In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import torch.nn as nn
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from soft_embedding import SoftEmbedding
from dataset import BiasDataset
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import pandas as pd

/opt/conda/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')


device(type='cpu')

In [3]:
def get_next_word_log_probabilities(model_input_1, model_input_2, model, logsoftmax):
    #model_input = create_input(sentence)
    #import pdb; pdb.set_trace()
    output_1 = model(**model_input_1)    
    next_word_logits_1 = output_1.logits[0][0][-1, :]
    next_word_log_probs_1 = logsoftmax(next_word_logits_1)
    
    output_2 = model(**model_input_2)   
    next_word_logits_2 = output_2.logits[0][0][-1, :]
    next_word_log_probs_2 = logsoftmax(next_word_logits_2)
    
    return next_word_log_probs_1, next_word_log_probs_2

In [4]:
n_tokens = 5
initialize_from_vocab = True
num_epochs = 2

In [5]:
dataset = BiasDataset("./data/occupation.csv", n_tokens = n_tokens)
dataloader = DataLoader(dataset = dataset, batch_size = 1,  shuffle = True, num_workers=2)

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
#freezing all the layers, and only keeping the tokens one active

for param in model.parameters():
    param.requires_grad_(False)
    
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-2)
logsoftmax = nn.LogSoftmax(dim = 0)
kl_loss = nn.KLDivLoss(reduction="batchmean", log_target=True)

In [8]:
epoch_loss = []
for epoch in range(2):
    le = 0
    batch_size = 64
    print(f'epoch : {epoch}')
    loss = 0
    curr_loss = []
    for data in dataloader:
        if not le%batch_size and le!=0:
            print(f'loss at batch : {le} is {loss.item()}')
            curr_loss.append(loss)
            loss = loss/batch_size
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss = 0
        
        log_probs1, log_probs2 = get_next_word_log_probabilities(data[0], data[1], model, logsoftmax)
        loss += kl_loss(log_probs1, log_probs2)
        le+=1
    epoch_loss.append(curr_loss)

epoch : 0
loss at batch : 64 is 0.0001957064087036997
loss at batch : 128 is 7.948162237880751e-05
loss at batch : 192 is 1.9660148609546013e-05
loss at batch : 256 is 1.0995300726790447e-05
loss at batch : 320 is 4.5175406739872415e-06
loss at batch : 384 is 3.454456646068138e-06
loss at batch : 448 is 3.3462060855526943e-06
loss at batch : 512 is 1.878013790701516e-06
loss at batch : 576 is 2.363196244914434e-06
loss at batch : 640 is 1.2759547871610266e-06
loss at batch : 704 is 1.1822346550616203e-06
loss at batch : 768 is 6.941272658878006e-07
loss at batch : 832 is 7.439597311531543e-07
loss at batch : 896 is 5.344198825696367e-07
loss at batch : 960 is 3.7122509866094333e-07
loss at batch : 1024 is 5.344933242668048e-07
loss at batch : 1088 is 3.5565102507462143e-07
loss at batch : 1152 is 2.936128282726713e-07
loss at batch : 1216 is 3.921564939446398e-07
loss at batch : 1280 is 2.9891413078075857e-07
loss at batch : 1344 is 3.063560427563061e-07
loss at batch : 1408 is 3.11456

RuntimeError: DataLoader worker (pid 27138) is killed by signal: Killed. 